# 問題35: 単語の出現頻度

形態素解析結果から単語の出現頻度を求め，出現頻度の高い順に並べよ．

In [ ]:
# 問題30で実装した関数を読み込む
import sys
sys.path.append('../..')  # 親ディレクトリをパスに追加
from collections import Counter
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# 日本語表示のための設定
plt.rcParams['font.family'] = 'IPAexGothic'

# 形態素解析結果を読み込む関数
def load_mecab_result(file_path):
    """
    MeCabの解析結果ファイルを読み込み、各形態素を辞書のリストとして返す関数
    
    Args:
        file_path (str): MeCab出力ファイルのパス
        
    Returns:
        list: 文のリスト。各文は形態素（辞書）のリスト
    """
    sentences = []
    current_sentence = []
    
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            # EOSは文の区切り
            if line == 'EOS\n':
                if current_sentence:
                    sentences.append(current_sentence)
                    current_sentence = []
                continue
                
            # 空行をスキップ
            if line == '\n':
                continue
                
            # タブで分割して表層形とそれ以外の情報に分ける
            try:
                surface, info = line.split('\t')
                
                # カンマで分割して品詞情報などを取得
                info_items = info.split(',')
                
                # 形態素情報を辞書として格納
                morpheme = {
                    'surface': surface,
                    'base': info_items[6],
                    'pos': info_items[0],
                    'pos1': info_items[1]
                }
                
                current_sentence.append(morpheme)
            except:
                # 不正な形式の行をスキップ
                continue
    
    # 最後の文が追加されていない場合に追加
    if current_sentence:
        sentences.append(current_sentence)
    
    return sentences

# 単語の出現頻度を計算する関数
def count_word_frequency(sentences):
    """
    形態素解析結果から単語の出現頻度を計算し、頻度順にソートする関数
    
    Args:
        sentences (list): 文のリスト。各文は形態素（辞書）のリスト
        
    Returns:
        list: (単語, 出現頻度)のタプルのリスト。出現頻度の降順でソート済み
    """
    # 全ての形態素の表層形を抽出
    words = [morpheme['surface'] for sentence in sentences for morpheme in sentence]
    
    # 単語の出現回数をカウント
    word_counts = Counter(words)
    
    # 出現頻度の降順でソート
    sorted_word_counts = sorted(word_counts.items(), key=lambda x: x[1], reverse=True)
    
    return sorted_word_counts

In [ ]:
# 形態素解析結果の読み込み
file_path = '../../data/neko.txt.mecab'
sentences = load_mecab_result(file_path)

# 単語の出現頻度を計算
word_frequencies = count_word_frequency(sentences)

# 結果の表示
print(f"異なり語数（ユニークな単語の数）: {len(word_frequencies)}")
print("\n出現頻度上位20語:")
for word, count in word_frequencies[:20]:
    print(f"{word}: {count}回")

# データフレームに変換して表示
df = pd.DataFrame(word_frequencies[:20], columns=['単語', '出現頻度'])
display(df)

# 出現頻度上位20語の棒グラフを作成
plt.figure(figsize=(12, 6))
plt.bar(df['単語'], df['出現頻度'], color='skyblue')
plt.title('出現頻度上位20語')
plt.xlabel('単語')
plt.ylabel('出現頻度')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

## 解説

この問題では、形態素解析結果から単語の出現頻度を計算し、頻度順にソートしました。

### 実装のポイント

1. **単語の抽出**: 全ての文から形態素の表層形を抽出しています。

2. **頻度のカウント**: Pythonの`collections.Counter`を使用して、効率的に単語の出現回数をカウントしています。

3. **ソート**: 出現頻度の降順でソートして、頻出語を上位に表示しています。

4. **可視化**: 出現頻度上位の単語を棒グラフで視覚化しています。

### 出現頻度の特徴

単語の出現頻度分布には、一般的に以下のような特徴があります：

1. **少数の高頻度語**: 少数の単語（主に助詞、助動詞などの機能語）が非常に高い頻度で出現します。

2. **多数の低頻度語**: 大多数の単語は低い頻度でしか出現しません。多くの単語は1回か2回しか出現しないことも珍しくありません。

3. **Zipfの法則**: 単語の出現頻度はその順位に反比例する傾向があります（問題39で詳しく扱います）。

### 結果の考察

出現頻度上位の単語を見ると、日本語の文章で一般的に多用される助詞（「の」「に」「は」など）や助動詞が多く含まれています。これらは文法的な機能を担う単語であり、どのような文章でも高頻度で出現します。

内容語（名詞、動詞、形容詞など）の中では、小説『吾輩は猫である』の特徴を反映した単語が上位に来ています。例えば、「猫」「主人」などの単語は、この小説の主題や登場人物に関連しています。

出現頻度の分析は、テキストの特徴や主題を把握するための基本的な手法です。より詳細な分析のためには、品詞ごとの頻度分析や、複合語（連語）の頻度分析なども有効です。